In [2]:
import pandas as pd

# Ler o arquivo Excel
df = pd.read_excel('systematik.xlsx')

# Classe Node
class Node:
    def __init__(self, name, url=None):
        self.name = name
        self.url = url
        self.children = []

    def add_child(self, child_node):
        self.children.append(child_node)

# Função para construir a árvore a partir do DataFrame
def build_tree_from_df(df):
    roots = {}
    for _, row in df.iterrows():
        root_name = row['Root']
        branch_name = row['Branch']
        sub_branch_name = row['Sub-Branch']
        leaf_name = row['Leaf']
        subleaf_name = row['Subleaf']
        url = row['URL']

        if root_name not in roots:
            roots[root_name] = Node(root_name)
        root = roots[root_name]

        branch = next((child for child in root.children if child.name == branch_name), None)
        if not branch:
            branch = Node(branch_name)
            root.add_child(branch)

        if pd.notna(sub_branch_name):
            sub_branch = next((child for child in branch.children if child.name == sub_branch_name), None)
            if not sub_branch:
                sub_branch = Node(sub_branch_name)
                branch.add_child(sub_branch)
        else:
            sub_branch = None

        if pd.notna(leaf_name):
            if sub_branch:
                leaf = next((child for child in sub_branch.children if child.name == leaf_name), None)
                if not leaf:
                    leaf = Node(leaf_name)
                    sub_branch.add_child(leaf)
            else:
                leaf = next((child for child in branch.children if child.name == leaf_name), None)
                if not leaf:
                    leaf = Node(leaf_name)
                    branch.add_child(leaf)
        else:
            leaf = None

        if pd.notna(subleaf_name):
            if leaf:
                subleaf = Node(subleaf_name, url)
                leaf.add_child(subleaf)
        elif leaf and pd.notna(url):
            leaf.url = url
        elif sub_branch and pd.notna(url):
            sub_branch.url = url
        elif branch and pd.notna(url):
            branch.url = url

    return list(roots.values())


# Função para gerar HTML completo da floresta
def generate_html(node):
    if node.url:
        name_html = f'<a href="{node.url}" target="_blank">{node.name}</a>'
    else:
        name_html = node.name

    if not node.children:
        return f'<li>{name_html}</li>'
    
    children_html = ''.join(generate_html(child) for child in node.children)
    return f'''
    <li>
        <span class="caret">{name_html}</span>
        <ul class="nested">
            {children_html}
        </ul>
    </li>
    '''

def generate_forest_html(forest):
    forest_html = ''.join(generate_html(root) for root in forest)
    return f'''
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Collapsible Tree Menu</title>
    <style>
        ul {{
            list-style-type: none; /* Entfernt die Aufzählungszeichen */
        }}
        li {{
            color: black; /* Setzt die Textfarbe auf Schwarz */
        }}
        li:hover {{
            color: #007c6c; /* Ändert die Textfarbe auf Grün Minerva bei Hover */
        }}
        .nested {{
            display: none;
        }}
        .active {{
            display: block;
        }}
        .caret {{
            cursor: pointer;
            user-select: none;
            font-weight: bold; /* Hebt die Caret-Items hervor */
        }}
        .caret::before {{
            content: "\25B6"; /* Rechts weisendes Dreieck */
            color: black;
            display: inline-block;
            margin-right: 6px;
        }}
        .caret-down::before {{
            content: "\25BC"; /* Nach unten weisendes Dreieck */
        }}
    </style>
</head>
<body>
    <h2>Collapsible Tree Menu</h2>
    <ul id="myMenu">
        {forest_html}
    </ul>
    <script>
        document.addEventListener('DOMContentLoaded', function() {{
            var toggler = document.getElementsByClassName("caret");
            for (var i = 0; i < toggler.length; i++) {{
                toggler[i].addEventListener("click", function() {{
                    this.parentElement.querySelector(".nested").classList.toggle("active");
                    this.classList.toggle("caret-down");
                }});
            }}
        }});
    </script>
</body>
</html>
    '''

# Construir a árvore a partir do DataFrame
forest = build_tree_from_df(df)

# Gerar conteúdo HTML
html_content = generate_forest_html(forest)

# Salvar o conteúdo HTML em um arquivo
with open('Test_6.html', 'w') as file:
    file.write(html_content)

print("HTML file 'Test_6.html' has been generated.")


FileNotFoundError: [Errno 2] No such file or directory: 'systematik.xlsx'